## Definitions

In [18]:
import random
import math
import cairo


def make_circ(x, y, r):
    return ('circ', x, y, r)

def make_rect(cx, cy, w, h):
    return ('rect', cx, cy, w, h)

def check_intersect(shape1, shape2):
    if shape1 is None or shape2 is None:
        return False
    if shape1[0] == 'circ' and shape2[0] == 'circ':
        s1, x1, y1, r1 = shape1
        s2, x2, y2, r2 = shape2
        return (x2 - x1) ** 2 + (y2 - y1) ** 2 <= (r1 + r2) ** 2
    if shape1[0] == 'rect' and shape2[0] == 'rect':
        s1, cx1, cy1, w1, h1 = shape1
        s2, cx2, cy2, w2, h2 = shape2
        return abs(cx1 - cx2) * 2 <= w1 + w2 and abs(cy1 - cy2) * 2 <= h1 + h2
    raise NotImplementedError()

def place_shape(placed_shapes, new_shape, max_r=10, trials=100, mode='circ'):
    ox, oy = new_shape[1], new_shape[2]
    candidate_shape = list(new_shape)
    for trial in range(trials):
        cur_r = (trial / trials) * 0.5 * max_r
        if mode == 'circ':
            cur_a = random.random() * math.pi * 2
            dx, dy = cur_r * math.cos(cur_a), cur_r * math.sin(cur_a)
        else:
            dx = cur_r * (random.random() * 2 - 1)
            dy = cur_r * (random.random() * 2 - 1)
        candidate_shape[1] = ox + dx
        candidate_shape[2] = oy + dy
        for other_shape in placed_shapes:
            if check_intersect(candidate_shape, other_shape):
                break
        else:
            return tuple(candidate_shape)
    return None

def place_shapes(shapes, max_r=10, trials=100, mode='circ'):
    random.seed(0)
    placed_shapes = []
    for shape in shapes:
        placed_shape = place_shape(placed_shapes, shape, max_r=max_r, trials=trials, mode=mode)
        placed_shapes.append(placed_shape)
    return placed_shapes

def shapes_bbox(shapes):
    x0 = y0 = float('inf')
    x1 = y1 = float('-inf')
    for shape in shapes:
        if shape is None:
            continue
        if shape[0] == 'rect':
            x, y, w, h = shape[1:]
        elif shape[0] == 'circ':
            x, y, r = shape[1:]
            w = h = 2 * r
        else:
            raise NotImplementedError()
        x0 = min(x0, x - w / 2)
        y0 = min(y0, y - h / 2)
        x1 = max(x1, x + w / 2)
        y1 = max(y1, y + h / 2)
    return x0, y0, x1 - x0, y1 - y0

def pad_bbox(bbox, pad):
    x, y, w, h = bbox
    return x - pad, y - pad, w + pad * 2, h + pad * 2

def set_font(ctx):
    ctx.select_font_face('Courier New', cairo.FONT_SLANT_NORMAL, cairo.FONT_WEIGHT_BOLD)

text_metrics = {}
text_metrics_ctx = cairo.Context(cairo.ImageSurface(cairo.FORMAT_ARGB32, 100, 100))
set_font(text_metrics_ctx)
text_metrics_ctx.set_font_size(100)

def get_text_width(text, size):
    if text not in text_metrics:
        print('recalc')
        tx = text_metrics_ctx.text_extents(text)
        w = tx.width
        text_metrics[text] = w
    return text_metrics[text] * size / 100

## Load data

In [14]:
from pathlib import Path

www_dir = Path('../../../hashquine.github.io/hicetnunc').resolve()
assert www_dir.is_dir()

thumbs_dir = www_dir / 'token_thumbs'

dataset_dir = Path('../../dataset').resolve()
assert dataset_dir.is_dir()

import json

addrs_ds = json.loads((dataset_dir / 'addrs.json').read_text('utf-8'))
tokens_ds = json.loads((dataset_dir / 'tokens.json').read_text('utf-8'))
sells_ds = json.loads((dataset_dir / 'sells.json').read_text('utf-8'))

from collections import Counter, defaultdict

## Compute layout

In [31]:
token_to_total_price = Counter()
token_to_sold_count = Counter()
author_to_tokens = defaultdict(set)
token_to_author = {}
token_to_ratio = {}

for sell in sells_ds.values():
    token_mint_date = tokens_ds[sell['token_id']]['mint_iso_date']
    sell_date = sell['tr_iso_date']

    if not ('2021-03-01T00:00:00Z' <= token_mint_date < '2021-04-01T00:00:00Z'):
        continue

    if sell['by_author']:
        token_to_total_price[sell['token_id']] += sell['price'] * sell['count']
        token_to_sold_count[sell['token_id']] += 1

    author_to_tokens[sell['author']].add(sell['token_id'])
    token_to_author[sell['token_id']] = sell['author']
    token_ds_entry = tokens_ds[sell['token_id']]

    width token_ds_entry['artifact_preview_width']
    token_ds_entry['artifact_preview_height']

object_id_order = [
    token_id
    for token_id, value in token_to_total_price.most_common()
]

In [23]:
shapes_order = [
    make_rect(0, 0, 100, 100),
] * 100
object_id_order = list(range(len(shapes_order)))
placed_shapes = place_shapes(shapes_order, max_r=1000, trials=3000, mode='rect')

## Draw

In [24]:
import cairo
import math
from io import BytesIO
from PIL import Image

bbox = pad_bbox(shapes_bbox(placed_shapes), 40)
scale_coeff = 1

WIDTH, HEIGHT = int(bbox[2] * scale_coeff), int(bbox[3] * scale_coeff)

surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, WIDTH, HEIGHT)
ctx = cairo.Context(surface)

ctx.scale(scale_coeff, scale_coeff)
ctx.translate(-bbox[0], -bbox[1])

ctx.set_source_rgb(0.0, 0.0, 0.0)
ctx.paint()

for object_id, shape in zip(object_id_order, placed_shapes):
    if shape is None:
        continue

    label = f'object_{object_id}'
    label_ref_width = get_text_width(label, 100)

    assert shape[0] == 'rect'
    x, y, w, h = shape[1:]
    ctx.save()
    ctx.new_path()
    ctx.translate(x - w / 2, y - h / 2)
    ctx.rectangle(0, 0, w, h)
    ctx.set_line_width(3.0)
    ctx.set_source_rgb(1.0, 0.0, 0.0)
    # ctx.stroke_preserve()

    fpath = thumbs_dir / (str(object_id + 200) + '.jpeg')
    if not fpath.exists():
        print(fpath, 'not exists')
        continue

    im = Image.open(fpath)
    buffer = BytesIO()
    im.save(buffer, format="PNG")
    buffer.seek(0)
    im_surface = cairo.ImageSurface.create_from_png(buffer)

    ctx.scale(w / im_surface.get_width(), h / im_surface.get_height())
    ctx.set_source_surface(im_surface)
    ctx.fill()
    ctx.restore()

    label_width = w * 0.9
    font_size = label_width / label_ref_width * 100

    ctx.new_path()
    set_font(ctx)
    ctx.move_to(x - label_width / 2, y + h / 2 - h * 0.05)
    ctx.set_font_size(font_size)
    ctx.text_path(label)
    ctx.set_source_rgb(0.0, 0.0, 0.0)
    ctx.stroke_preserve()
    ctx.set_line_width(5)
    ctx.set_source_rgb(1.0, 1.0, 1.0)
    ctx.fill()

surface.write_to_png("top_commenters_visualization.png")

D:\PycharmProjects\hashquine.github.io\hicetnunc\token_thumbs\237.jpeg not exists
